In [138]:
import json
from web3 import Web3
from web3.middleware import geth_poa_middleware
import pickle
import requests
import pandas as pd
from datetime import datetime
import numpy as np
import math

In [139]:
web3 = Web3(Web3.HTTPProvider("https://bsc-dataseed.binance.org/"))
web3.middleware_onion.inject(geth_poa_middleware, layer=0)
web3.isConnected()

True

In [188]:
# Address = token contract
apikey = "GKDPH6RRX5YSH1TYPMNHNVKGMB6I9V3I8T"
address = "0xF700D4c708C2be1463E355F337603183D20E0808"

# Methods
approval_method = "0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925"
transfer_method = "0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef"
stake_method = "0xe1fffcc4923d04b559f4d29a8bfc6cda04eb5b0d3c460751c2402c5c5cc9109c"


# Set block
currentblock = web3.eth.blockNumber
endblock = currentblock

# Set topic0
topic0 = approval_method

# Current block
currentblock = web3.eth.blockNumber
endblock = currentblock

# Otros contratos
claim_contract = "0x739e6e290f9aa6ef3a929950aa47795b1d187229"
claim_venture = "0x2794A45B10113D2bd7ED28AF677077c75bB308D0"
stake_contract = "0xdf5c4c1b98a4d045573ab2005d77e87daa58c853"


# Liquidity Pool
pcs = "0x00000000000000000000000072121d60b0e2f01c0fb7fe32ca24021b42165a40"
pcs_address = "0x72121d60b0e2F01c0FB7FE32cA24021b42165A40"

ape = "0x000000000000000000000000cC3A3Bc1d76Df321f94716E88224638C439267aa"
ape_address = "0xcC3A3Bc1d76Df321f94716E88224638C439267aa"

biswap = "0x000000000000000000000000B4c88543Bd95617Fb61F3d031B8E8E19aF33201E"
biswap_address = "0xB4c88543Bd95617Fb61F3d031B8E8E19aF33201E"

# Set block
currentblock = web3.eth.blockNumber
endblock = currentblock

# Set topic0
topic0 = approval_method

# Wallets to check
check_wallets = [
    address,
    claim_contract,
    stake_contract,
    pcs_address,
    ape_address,
    biswap_address,
]

In [141]:
# Get ABI

url = "https://api.bscscan.com/api"
params = {
    "module": "contract",
    "action": "getabi",
    "address": address,
    "apikey": apikey,
}

response = requests.get(url, params=params)

if response.status_code == 200:
    abi = response.json()["result"]
    print("Abi recibida correctamente")
else:
    print("Error en la solicitud")

Abi recibida correctamente


In [142]:
w3address = Web3.toChecksumAddress(address.lower())
contract = web3.eth.contract(address=w3address, abi=abi)

token = contract.functions.name().call()
symbol = contract.functions.symbol().call()

print(f"Abi del token: {token}, con simbolo: {symbol}, recibida correctamente")

Abi del token: Galactic Quadrant, con simbolo: GQ, recibida correctamente


In [143]:
# Get "Internal Transactions" by Block Range
def get_internal_transactions(endblock, numberofblocks):
    url = "https://api.bscscan.com/api"
    params = {
        "module": "account",
        "action": "txlistinternal",
        "startblock": endblock - numberofblocks,
        "endblock": endblock,
        "page": 1,
        "offset": 10,
        "sort": "asc",
        "apikey": apikey,
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        transactions = response.json()["result"]
        return transactions
    else:
        return "Error en la solicitud"


# Get Logs by Address sin topic0
def get_logs_loop(address, endblock, blocks):
    if blocks % 1000 != 0:
        raise ValueError("El número de bloques debe ser múltiplo de 1000")
    total_logs = []
    for loop in range(int(blocks / 1000)):
        logs = get_logs1000(address, endblock - loop * 1000)
        total_logs = total_logs + logs
    return total_logs


def get_logs1000(address, endblock):
    url = "https://api.bscscan.com/api"
    params = {
        "module": "logs",
        "action": "getLogs",
        "fromBlock": endblock - 1000,
        "toBlock": endblock,
        "address": address,
        "apikey": apikey,
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        logs = response.json()["result"]
        return logs
    else:
        return "Error en la solicitud"

# Get all logs
def get_all_logs_loop(address, endblock, fromblock):
    blocks = endblock-fromblock
    # if blocks % 1000 != 0:
    #     raise ValueError("El número de bloques debe ser múltiplo de 1000")
    total_logs = []
    for loop in range(int(blocks / 1000)):
        logs = get_logs1000(address, fromblock + loop * 1000)
        if logs is not None:
            total_logs = total_logs + logs
    return total_logs


def get_all_logs1000(address, fromblock):
    url = "https://api.bscscan.com/api"
    params = {
        "module": "logs",
        "action": "getLogs",
        "fromBlock": fromblock,
        "toBlock": fromblock + 1000,
        "address": address,
        "apikey": apikey,
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        logs = response.json()["result"]
        return logs
    else:
        return "Error en la solicitud"


# Get Logs by Address
def get_logstopic_loop(address, endblock, blocks, topic0):
    if blocks % 1000 != 0:
        raise ValueError("El número de bloques debe ser múltiplo de 1000")
    total_logs = []
    for loop in range(int(blocks / 1000)):
        logs = get_logs1000(address, endblock - loop * 1000)
        total_logs = total_logs + logs
    return total_logs


def get_logs1000topic(address, endblock, topic0):
    url = "https://api.bscscan.com/api"
    params = {
        "module": "logs",
        "action": "getLogs",
        "fromBlock": endblock - 1000,
        "toBlock": endblock,
        "topic0": topic0,
        "address": address,
        "apikey": apikey,
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        logs = response.json()["result"]
        return logs
    else:
        return "Error en la solicitud"


# Get Logs by Address and From
def get_logstopic0_1_loop(address, endblock, blocks, topic0, topic1):
    if blocks % 1000 != 0:
        raise ValueError("El número de bloques debe ser múltiplo de 1000")
    total_logs = []
    for loop in range(int(blocks / 1000)):
        logs = get_logs1000(address, endblock - loop * 1000)
        total_logs = total_logs + logs
    return total_logs


def get_logs1000topic0_1(address, endblock, topic0, topic1):
    url = "https://api.bscscan.com/api"
    params = {
        "module": "logs",
        "action": "getLogs",
        "fromBlock": endblock - 1000,
        "toBlock": endblock,
        "topic0": topic0,
        "topic0_1": "and",
        "topic1": topic1,
        "address": address,
        "apikey": apikey,
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        logs = response.json()["result"]
        return logs
    else:
        return "Error en la solicitud"


# Get a list of 'BEP-20 Token Transfer Events' by wallet
def get_token_transfers(wallet, address):
    url = "https://api.bscscan.com/api"
    params = {
        "module": "account",
        "action": "tokentx",
        "contractaddress": address,
        "address": wallet,
        "page": 1,
        "offset": 1000,
        "startblock": 0,
        "endblock": 999999999,
        "sort": "asc",
        "apikey": apikey,
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        transactions = response.json()["result"]
        return transactions
    else:
        return "Error in the request"


# Remove known wallets
def delete_wallet_from_df(df, check_wallets):
    for wallet_address in check_wallets:
        result = df["wallets"].isin([wallet_address.lower()]).any()
        print(f"Esta {wallet_address} en las wallets? {result}")
        if result:
            df = df.query('wallets != "{}"'.format(wallet_address.lower()))
    return df

### 1.3 Methods
Scan de transacciones para identificar los diferentes methods del contrato, para más tarde usarlos para filtrar

In [144]:
## Extractor de Events/topics0
total_logs = get_logs_loop(address, endblock, 100000)


# Save hash in a list
total_hash = []
for i in range(len(total_logs)):
    event_hash = total_logs[i]["topics"][0]
    total_hash.append(event_hash)

unique_event_hash = list(set(total_hash))


# Cantidad de logs extraidos
print(
    f"Se han obtenido {len(total_logs)} logs, en los cuales hay {len(unique_event_hash)} event hash unicos."
)
print(unique_event_hash)


# Guarda los nuevos event hash
logs_unique = []
for value in unique_event_hash:
    if value not in logs_unique:
        logs_unique.append(value)

print(
    f"Actualmente hay almacenados {len(logs_unique)} event hash unicos que utilizaremos para reconocer los diferentes methods de topic0"
)

Se han obtenido 12204 logs, en los cuales hay 3 event hash unicos.
['0x9d228d69b5fdb8d273a2336f8fb8612d039631024ea9bf09c424a9503aa078f0', '0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925', '0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef']
Actualmente hay almacenados 3 event hash unicos que utilizaremos para reconocer los diferentes methods de topic0


### 2.1 Logs Data
Recopilar toda la data de las interacciones con el contrato

In [171]:
# Setting up blocks
currentblock = web3.eth.blockNumber
endblock = currentblock 
fromblock = 15789191

# Get data
logs_weekend = get_all_logs_loop(address, endblock, fromblock)
token_tx = pd.DataFrame(logs_weekend)
token_tx.head()

,address,topics,data,blockNumber,blockHash,timeStamp,gasPrice,gasUsed,logIndex,transactionHash,transactionIndex
0,0xf700d4c708c2be1463e355f337603183d20e0808,"[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef, 0x0000000000000000000000007...",0x0000000000000000000000000000000000000000000001056126f0307b04e000,0xf0e8a7,0xd529e97438469961d622b8332270f477aac0b79a693d5681197f574fa7551a03,0x622312a1,0x12a05f200,0x1d7b3,0x1e0,0x80428376e0e0463c9a7108a191ede99fbb55dfc00bf418c0572818899235f1c6,0xb5
1,0xf700d4c708c2be1463e355f337603183d20e0808,"[0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925, 0x0000000000000000000000007...",0xffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff,0xf0e8cf,0xe2833a64bf708351148103a75810a5174e253236b499ae0b13fd08209a4ba1fc,0x62231319,0x12a05f200,0xadf4,0x96,0x7a510d0d97b21dd4636aad0c09bc0169a5dab7940fd17fb4eaca060de7aef80d,0x45
2,0xf700d4c708c2be1463e355f337603183d20e0808,"[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef, 0x000000000000000000000000e...",0x0000000000000000000000000000000000000000000000c73c56f2b0e2650000,0xf0e8d2,0x39493187fde9893db8837c090b72ffb9701933de16b9fe8a83e200d9e3939304,0x62231322,0x12a05f200,0xfd8b,0x16c,0x6bfea2bcfc8885a6185eb381e6d83473b7ad1b7134707428e3faec9920c818d0,0xa4
3,0xf700d4c708c2be1463e355f337603183d20e0808,"[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef, 0x0000000000000000000000007...",0x00000000000000000000000000000000000000000000003635c9adc5dea00000,0xf0e8de,0x8ee597e414ca733b1162c5c7312f9a39988b551b21a050c749fc5b97b8fbd775,0x62231346,0x12a05f200,0x23614,0x14c,0x697eb1089ef04f80b710ac5eaa266d330be58805baa9e19488db203a2df6e408,0x68
4,0xf700d4c708c2be1463e355f337603183d20e0808,"[0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925, 0x0000000000000000000000007...",0xffffffffffffffffffffffffffffffffffffffffffffffc9ca36523a215fffff,0xf0e8de,0x8ee597e414ca733b1162c5c7312f9a39988b551b21a050c749fc5b97b8fbd775,0x62231346,0x12a05f200,0x23614,0x14d,0x697eb1089ef04f80b710ac5eaa266d330be58805baa9e19488db203a2df6e408,0x68


In [172]:
token_tx.to_csv("csv/gq_all_logs_raw.csv", index=False)

#### 2.1.1 Cleaning
Limpiamos la data y obtenemos todas las wallets que interactuan con el contrato para más tarde extraer sus transacciones

In [173]:
# TOKEN LOG CLEANING

# Data Clean
token_tx["timeStamp"] = token_tx["timeStamp"].apply(lambda x: int(x, 16))
token_tx["timeStamp"] = token_tx["timeStamp"].apply(lambda x: datetime.fromtimestamp(x))

df_topics = pd.DataFrame(token_tx["topics"].tolist(), columns=["method", "from", "to"])
token_tx = pd.concat([token_tx, df_topics], axis=1)

token_tx = token_tx.drop(
    columns=[
        "topics",
        "address",
        "data",
        "blockNumber",
        "blockHash",
        "gasPrice",
        "gasUsed",
        "logIndex",
        "gasUsed",
        "transactionIndex",
    ],
    axis=1,
)

token_tx.sort_values(by="timeStamp", ascending=True, inplace=True)

# Filtrar la fecha
token_tx["intime"] = 0
for index, row in token_tx.iterrows():
    if row["timeStamp"].dayofyear < 31 and row["timeStamp"].dayofyear > 28:
        token_tx.at[index, "intime"] = 1

token_tx.head()

token_tx["intime"] = token_tx["intime"].astype(bool)
token_tx = token_tx[token_tx["intime"] == 1]

token_tx = token_tx.drop(
    columns=["intime"],
    axis=1,
)

# Standarizar address, from & to
token_tx["to"] = "0x" + token_tx["to"].str[26:]
token_tx["from"] = "0x" + token_tx["from"].str[26:]

# Filtrar solo transferencias
token_tx = token_tx[token_tx["method"] == transfer_method]

In [174]:
token_tx.to_csv("csv/gq_all_logs.csv", index=False)
token_tx.head()

,timeStamp,transactionHash,method,from,to
808150,2023-01-29 00:04:18,0x6b7ca16a1dc279b6540ba156557714ce4e13ee986522813aa6f463f90d8f1fb6,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef,0xb88d09175676913d69924acac17ec69037f1551e,0x4571d47e9ed7d838e9af02ef7ae3f54c56650a40
808151,2023-01-29 00:05:18,0x0cc6b78ba8e98cfc7b562b215caf886673f75adc6df7c99b66cafd0aac8d9b61,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef,0xb4c88543bd95617fb61f3d031b8e8e19af33201e,0xf1ec5c64da312f96aef027c3dd26880b4e1a59b6
808152,2023-01-29 00:05:18,0xe7fc044ab8530a0c6ede611ad2b4fa499db181c57c693dfca3739219fb5466fd,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef,0x72121d60b0e2f01c0fb7fe32ca24021b42165a40,0xb4c88543bd95617fb61f3d031b8e8e19af33201e
808153,2023-01-29 00:05:18,0x5cab2d07852db057169120f1f42ad0e5ef912ab0bd06e69e4dc2103fc57a899b,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef,0xcc3a3bc1d76df321f94716e88224638c439267aa,0xef713415d4c058cb3cd5224174e28c9384d08e21
808154,2023-01-29 00:05:18,0x5cab2d07852db057169120f1f42ad0e5ef912ab0bd06e69e4dc2103fc57a899b,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef,0xef713415d4c058cb3cd5224174e28c9384d08e21,0xb4c88543bd95617fb61f3d031b8e8e19af33201e


In [175]:
wallets_1 = list(token_tx["from"].unique())
len(wallets_1)

992

In [176]:
wallets_2 = list(token_tx["to"].unique())
len(wallets_2)

1393

In [177]:
wallets = wallets_1 + wallets_2
wallets = pd.DataFrame(wallets, columns=["wallets"])
wallets = wallets.drop_duplicates()
wallets.shape

(1694, 1)

In [178]:
wallets = delete_wallet_from_df(wallets, check_wallets)

Esta 0xF700D4c708C2be1463E355F337603183D20E0808 en las wallets? False
Esta 0x739e6e290f9aa6ef3a929950aa47795b1d187229 en las wallets? True
Esta 0xdf5c4c1b98a4d045573ab2005d77e87daa58c853 en las wallets? True
Esta 0x72121d60b0e2F01c0FB7FE32cA24021b42165A40 en las wallets? True
Esta 0xcC3A3Bc1d76Df321f94716E88224638C439267aa en las wallets? True
Esta 0xB4c88543Bd95617Fb61F3d031B8E8E19aF33201E en las wallets? True


In [179]:
wallets = delete_wallet_from_df(wallets, check_wallets)

Esta 0xF700D4c708C2be1463E355F337603183D20E0808 en las wallets? False
Esta 0x739e6e290f9aa6ef3a929950aa47795b1d187229 en las wallets? False
Esta 0xdf5c4c1b98a4d045573ab2005d77e87daa58c853 en las wallets? False
Esta 0x72121d60b0e2F01c0FB7FE32cA24021b42165A40 en las wallets? False
Esta 0xcC3A3Bc1d76Df321f94716E88224638C439267aa en las wallets? False
Esta 0xB4c88543Bd95617Fb61F3d031B8E8E19aF33201E en las wallets? False


In [180]:
wallets.to_csv("csv/gq_all_wallets.csv", index=False)

### 2.2 Transacciones Data
A traves de las wallets obtenidas sacamos todas sus transacciones que interactuen con el contrato para obtener más información de estas.

In [212]:
#Iterate buscando transactions en todas las wallets.
total_tx = []
for index, row in wallets.iterrows():
    wallet = row['wallets']
    wallets_tx = get_token_transfers(wallet, address)
    total_tx = total_tx + wallets_tx 

total_tx = pd.DataFrame(total_tx)
total_tx.to_csv("csv/gq_all_tx_raw.csv", index=False)
total_tx.head()


,blockNumber,timeStamp,hash,nonce,blockHash,from,contractAddress,to,value,tokenName,tokenSymbol,tokenDecimal,transactionIndex,gas,gasPrice,gasUsed,cumulativeGasUsed,input,confirmations
0,23499109,1669801290,0x989ae51448d5b7cbaada8f644e8ba1788f906278ce35b1352cf42070160033da,1380,0x0fa35d69077a64182afcaede941a694e8080b3c10e87399d488a12286eb5f7e9,0xb6af19068482ec5feb4c4ef7f81a4dfdfa0bbad2,0xf700d4c708c2be1463e355f337603183d20e0808,0xb88d09175676913d69924acac17ec69037f1551e,3335881575391497601387464,Galactic Quadrant,GQ,18,85,85050,5000000000,41700,7061991,deprecated,1935213
1,23500052,1669804123,0xd4b49a4fed7db80e08f33e80b88519fa73da0d52fb3a04b430888aa78f8fabca,147,0x9e7699e6d84c3f2016b39b04d9590c55010a07785bc01e4cb9e1cd668e7c8a15,0x170e7d670cd9260a70bcf00d915e05983d040305,0xf700d4c708c2be1463e355f337603183d20e0808,0xb88d09175676913d69924acac17ec69037f1551e,840567311251888613858,Galactic Quadrant,GQ,18,150,112035,5000000000,91343,24212646,deprecated,1934270
2,23500078,1669804201,0x0905c8c0151f9c34bd7e2f352f812eadff343151610bee49e466f0b282c98dac,150,0xb8fc498e4b01e568374a90bb50a8a372db136eb388193c1c723afc2406e40858,0x170e7d670cd9260a70bcf00d915e05983d040305,0xf700d4c708c2be1463e355f337603183d20e0808,0xb88d09175676913d69924acac17ec69037f1551e,3994520037709685716394901,Galactic Quadrant,GQ,18,120,85682,5000000000,64990,11383261,deprecated,1934244
3,23500661,1669805976,0x6eaf5e484343457b07233ebb8ce192daacb4f36174b6aeca8c048af220c725b3,151,0xabeb596399f71b4c67c1be867e964f8eb2e176452ef43a7a6e2b10c8961e5a7e,0xb88d09175676913d69924acac17ec69037f1551e,0xf700d4c708c2be1463e355f337603183d20e0808,0x170e7d670cd9260a70bcf00d915e05983d040305,3995360605020937605008759,Galactic Quadrant,GQ,18,274,90503,5000000000,54813,24749185,deprecated,1933661
4,23501083,1669807242,0x7ed5efda378ef24b9fb19a018244b3b98ada10dd60ac0698b636cc014d5b88bb,18,0x874a1571f081caed06ce2d51f27a667b304aa2afaf46a932e75b81a95a1f09eb,0x371848dd23d33bc615ba1e6d53045e5cecd48a43,0xf700d4c708c2be1463e355f337603183d20e0808,0xb88d09175676913d69924acac17ec69037f1551e,67116520258622566489516,Galactic Quadrant,GQ,18,37,168070,5000000000,91355,4152824,deprecated,1933239


In [213]:
total_tx.shape

(114965, 19)

#### 2.1.1 Cleaning
Limpiamos la data para poder repartir las transacciones en los diferentes tipos, compras, stakes, claims...

In [214]:
# Data Clean
token_tx = total_tx
token_tx['timeStamp'] = pd.to_datetime(token_tx['timeStamp'], unit='s')
token_tx['timeStamp'] = token_tx['timeStamp'].dt.strftime('%Y-%m-%d %H:%M:%S')

# df_topics = pd.DataFrame(token_tx["topics"].tolist(), columns=["method", "from", "to"])
# token_tx = pd.concat([token_tx, df_topics], axis=1)

token_tx = token_tx.drop(
    columns=[
        "blockNumber",
        "nonce",
        "blockHash",
        "contractAddress",
        "tokenName",
        "tokenSymbol",
        "tokenDecimal",
        "transactionIndex",
        "gas",
        "gasPrice",
        "gasUsed",
        "cumulativeGasUsed",
        "input",
        "confirmations",        
    ],
    axis=1,
)

token_tx.sort_values(by="timeStamp", ascending=True, inplace=True)

In [215]:
token_tx['mont'] = round(token_tx['value'].astype(float)/1000000000000000000, 2)

token_tx = token_tx.drop(columns=['value'], axis=1)

token_tx.rename(columns={'mont' : 'value'}, inplace=True)

token_tx.sort_values(by='timeStamp', ascending=True, inplace=True)

#Staking Check
token_tx['staking'] = ((token_tx['to'] == stake_contract) | (token_tx['from'] == stake_contract)).astype(int)

token_tx['stake_balance'] = 0
for index, row in token_tx.iterrows():
    if row['staking'] == 1:
        if row['to'] == wallet.lower():
            token_tx.at[index, 'stake_balance'] = row['value']
        else:
            token_tx.at[index, 'stake_balance'] = -row['value']

In [216]:
token_tx.to_csv("csv/gq_all_tx.csv", index=False)
token_tx.head()

,timeStamp,hash,from,to,value,staking,stake_balance
110336,2022-03-01 10:26:27,0xbc0e2dabb9084e3ef977386835c6a88c6d0ba1c3624318af5d4c56472b2f2871,0x0000000000000000000000000000000000000000,0xb6af19068482ec5feb4c4ef7f81a4dfdfa0bbad2,1.000000e+09,0,0.0
21151,2022-03-02 21:19:13,0x811110e4606e00cc8389b33b88bf3383eb9682f007e21c0a4227721cbfe71433,0x0fb3e3f4f157580e81deaf697b1b41d0a202d717,0x2794a45b10113d2bd7ed28af677077c75bb308d0,1.250000e+06,0,0.0
21152,2022-03-03 04:19:16,0x84b0c59ca63c4f8fa63184b7d28dc5295d79a0618822b093b58340cd748e5358,0x2794a45b10113d2bd7ed28af677077c75bb308d0,0x9044b7e45eabd1ca456ac5072d12a7e258cd27f1,1.250000e+06,0,0.0
21153,2022-03-03 04:22:13,0xaa1873fc06cd0c8cd0dbbc68c3f08ca6fa86cc15ecc77791612937e2ffebbec2,0x9044b7e45eabd1ca456ac5072d12a7e258cd27f1,0x2794a45b10113d2bd7ed28af677077c75bb308d0,1.250000e+06,0,0.0
25882,2022-03-03 08:37:52,0xa0bcb77b5e5d4a6c92d0dbd190934750f2d3e96676e68dc8a20c3278e5729978,0x2699eacc1cd8d115c36c72d9ce075cfb69a9dc37,0x02d51353a4d8d115350af5f60917570c3faf7a83,2.625000e+06,0,0.0


In [217]:
#Balance Staked
print(f"Actualmente tiene {token_tx['stake_balance'].sum()} tokens stakeados")

#Tokens holds
print(f"y holdea {move_balance.sum() + token_tx['stake_balance'].sum()} tokens.")

Actualmente tiene -495781149.7199999 tokens stakeados
y holdea -17422795222.270004 tokens.


In [218]:
stake_tx = token_tx[(token_tx["staking"] == 1)]
stake_tx.to_csv("csv/gq_all_stake_tx.csv", index=False)
stake_tx.head()

,timeStamp,hash,from,to,value,staking,stake_balance
53006,2022-05-30 16:58:47,0x1d00f31e32fdec1c56bb80a8e4f8979aa3da6ce7dcf9a28eb5fc103695469a98,0x19c20d36c02a1a27125d638f3c16d04aaff2ceae,0xdf5c4c1b98a4d045573ab2005d77e87daa58c853,42557.06,1,-42557.06
52799,2022-05-30 17:04:19,0x45c4ea5fbeebd22ac4027744f69de6e4d9dedd6bd02e0f5976a2e3b1c65e0581,0xd9468f7bab04d6b306e9229aae69462a3be2ea5b,0xdf5c4c1b98a4d045573ab2005d77e87daa58c853,13410.00,1,-13410.00
54112,2022-05-30 17:04:49,0x58e93cca984f193b020793cd223c91666587743ba7547b3f34bfc58e719fa13d,0xb956cafc8247f5ce2b6642a4355eda020715ce84,0xdf5c4c1b98a4d045573ab2005d77e87daa58c853,34500.00,1,-34500.00
52801,2022-05-30 17:07:23,0x534bc817d17a43a1fdeffd4a89da1a950772f63daa0f6754e14a08e3d962af86,0xd9468f7bab04d6b306e9229aae69462a3be2ea5b,0xdf5c4c1b98a4d045573ab2005d77e87daa58c853,216000.00,1,-216000.00
90763,2022-05-30 17:08:11,0x21c74737696a6b872e1fb58571a5ee064c34a791e82243e1cd057f4b3e26300f,0xd1131556881f6b04a20d11e33048c89554c422c8,0xdf5c4c1b98a4d045573ab2005d77e87daa58c853,61578.59,1,-61578.59


In [219]:
claim_tx = token_tx[(token_tx["from"] == claim_contract) | (token_tx["from"] == claim_venture)]
claim_tx.to_csv("csv/gq_all_claim_tx.csv", index=False)
claim_tx.head()

,timeStamp,hash,from,to,value,staking,stake_balance
99869,2022-03-03 15:25:42,0x9c93a3ff26be962c9bf00b82770f2073cf6d894cb5245a5647c4b8267f967ea8,0x739e6e290f9aa6ef3a929950aa47795b1d187229,0x37efe04c673be382de963d89da77f530bce37808,3272.84,0,0.0
99761,2022-03-03 15:25:45,0x5a9c4c80867c113f9cbf71e78718db3e502a6c8f0f122e68082d26f73bf4da0b,0x739e6e290f9aa6ef3a929950aa47795b1d187229,0x4d90f04b712f16ded9289da63b931fb929a15d60,1022.76,0,0.0
100412,2022-03-03 15:25:51,0x4fe747de23903f4ff1224bf24d8c1527bb9a6f42e5d2d571669c5b71a9f253c7,0x739e6e290f9aa6ef3a929950aa47795b1d187229,0xa24bcda5499bc32f75020755772f4dc038b69579,2337.75,0,0.0
111222,2022-03-03 15:25:51,0x917af70a5f29d888bf99b4889868bcc6046686d84626fa51b3c8dd701cce5615,0x739e6e290f9aa6ef3a929950aa47795b1d187229,0xfc146c248b29e61a47ea934c3c9e786bd104f1d9,730.55,0,0.0
103642,2022-03-03 15:26:12,0x4628e3aa93dff7c8e3762ee27c2ce34a1ac997fba966d2110b9dfecf99cb7b25,0x739e6e290f9aa6ef3a929950aa47795b1d187229,0xcc43c0cef2afd62a0d89850215ee72656ec7c261,730.55,0,0.0


In [232]:
#Boolean para comprobar que interactuan con liquidity pools
token_tx["pcs_address"] = 0
token_tx["ape_address"] = 0
token_tx["biswap_address"] = 0


# Loop sobre el DataFrame
for i, row in token_tx.iterrows():
    if row["from"] == pcs_address.lower():
        token_tx.at[i, "pcs_address"] = 1
    elif row["from"] == ape_address.lower():
        token_tx.at[i, "ape_address"] = 1
    elif row["from"] == biswap.lower():
        token_tx.at[i, "biswap_address"] = 1
    elif row["to"] == pcs_address.lower():
        token_tx.at[i, "pcs_address"] = 1
    elif row["to"] == ape_address.lower():
        token_tx.at[i, "ape_address"] = 1
    elif row["to"] == biswap.lower():
        token_tx.at[i, "biswap_address"] = 1

token_tx.head()

,timeStamp,hash,from,to,value,staking,stake_balance,pcs_address,ape_address,biswap_address
110336,2022-03-01 10:26:27,0xbc0e2dabb9084e3ef977386835c6a88c6d0ba1c3624318af5d4c56472b2f2871,0x0000000000000000000000000000000000000000,0xb6af19068482ec5feb4c4ef7f81a4dfdfa0bbad2,1.000000e+09,0,0.0,0,0,0
21151,2022-03-02 21:19:13,0x811110e4606e00cc8389b33b88bf3383eb9682f007e21c0a4227721cbfe71433,0x0fb3e3f4f157580e81deaf697b1b41d0a202d717,0x2794a45b10113d2bd7ed28af677077c75bb308d0,1.250000e+06,0,0.0,0,0,0
21152,2022-03-03 04:19:16,0x84b0c59ca63c4f8fa63184b7d28dc5295d79a0618822b093b58340cd748e5358,0x2794a45b10113d2bd7ed28af677077c75bb308d0,0x9044b7e45eabd1ca456ac5072d12a7e258cd27f1,1.250000e+06,0,0.0,0,0,0
21153,2022-03-03 04:22:13,0xaa1873fc06cd0c8cd0dbbc68c3f08ca6fa86cc15ecc77791612937e2ffebbec2,0x9044b7e45eabd1ca456ac5072d12a7e258cd27f1,0x2794a45b10113d2bd7ed28af677077c75bb308d0,1.250000e+06,0,0.0,0,0,0
25882,2022-03-03 08:37:52,0xa0bcb77b5e5d4a6c92d0dbd190934750f2d3e96676e68dc8a20c3278e5729978,0x2699eacc1cd8d115c36c72d9ce075cfb69a9dc37,0x02d51353a4d8d115350af5f60917570c3faf7a83,2.625000e+06,0,0.0,0,0,0


In [233]:
#Filtrar columnas que interactuan con alguna liquidity pool y el vesting
liquidity_tx = token_tx[(token_tx["pcs_address"] != 0) | (token_tx["ape_address"] != 0) | (token_tx["biswap_address"] != 0)]

#Add balance 
liquidity_tx['balance'] = 0
liquidity_tx['balance'] = np.where((liquidity_tx['from'] == pcs_address.lower()) | (liquidity_tx['from'] == ape_address.lower()) | (liquidity_tx['from'] == biswap.lower()), liquidity_tx['value'], -liquidity_tx['value'])

liquidity_tx.head()


C:\Users\progr\AppData\Local\Temp\ipykernel_8652\42246841.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  liquidity_tx['balance'] = 0
C:\Users\progr\AppData\Local\Temp\ipykernel_8652\42246841.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  liquidity_tx['balance'] = np.where((liquidity_tx['from'] == pcs_address.lower()) | (liquidity_tx['from'] == ape_address.lower()) | (liquidity_tx['from'] == biswap.lower()), liquidity_tx['value'], -liquidity_tx['value'])


,timeStamp,hash,from,to,value,staking,stake_balance,pcs_address,ape_address,biswap_address,balance
16216,2022-03-03 15:25:39,0x83dd0b0498d75eddb297801a7cb5bf2156b8e7f676dd656b9d8cf414bb050d1f,0x72121d60b0e2f01c0fb7fe32ca24021b42165a40,0x0c9064761eac5972f53d199fdd3bc6391707c5a2,7.34,0,0.0,1,0,0,7.34
106881,2022-03-03 15:25:39,0xc1919bfc7172d4d46b682165a888481ff77e5e4714c2d3fa22adb92752a9cf0a,0x72121d60b0e2f01c0fb7fe32ca24021b42165a40,0xff288ca8ebeddaee16f158202a8e93cbfefc4213,34366.47,0,0.0,1,0,0,34366.47
16234,2022-03-03 15:25:45,0xe8f09f09102f18db097b54198dc8bd0b7a4380c07c07464eadbe48250cc886c2,0x72121d60b0e2f01c0fb7fe32ca24021b42165a40,0x0c9064761eac5972f53d199fdd3bc6391707c5a2,6.98,0,0.0,1,0,0,6.98
16230,2022-03-03 15:25:45,0x2caf9244952f121fb55689b41cf0c73e3c1dea37a708bc8d3807261c73d4736d,0x72121d60b0e2f01c0fb7fe32ca24021b42165a40,0x0c9064761eac5972f53d199fdd3bc6391707c5a2,6.97,0,0.0,1,0,0,6.97
16226,2022-03-03 15:25:45,0x2b83431e7f95d9030eaa39faab569d6635d10db666cbb1e8200bb44e0472c341,0x72121d60b0e2f01c0fb7fe32ca24021b42165a40,0x0c9064761eac5972f53d199fdd3bc6391707c5a2,6.95,0,0.0,1,0,0,6.95


In [234]:
liquidity_tx = liquidity_tx.drop(
    columns=[
        "pcs_address",
        "ape_address",
        "biswap_address",
        "stake_balance",
        "staking",   
    ],
    axis=1,
)

liquidity_tx.to_csv("csv/gq_all_liquidity_tx.csv", index=False)
liquidity_tx.head()


,timeStamp,hash,from,to,value,balance
16216,2022-03-03 15:25:39,0x83dd0b0498d75eddb297801a7cb5bf2156b8e7f676dd656b9d8cf414bb050d1f,0x72121d60b0e2f01c0fb7fe32ca24021b42165a40,0x0c9064761eac5972f53d199fdd3bc6391707c5a2,7.34,7.34
106881,2022-03-03 15:25:39,0xc1919bfc7172d4d46b682165a888481ff77e5e4714c2d3fa22adb92752a9cf0a,0x72121d60b0e2f01c0fb7fe32ca24021b42165a40,0xff288ca8ebeddaee16f158202a8e93cbfefc4213,34366.47,34366.47
16234,2022-03-03 15:25:45,0xe8f09f09102f18db097b54198dc8bd0b7a4380c07c07464eadbe48250cc886c2,0x72121d60b0e2f01c0fb7fe32ca24021b42165a40,0x0c9064761eac5972f53d199fdd3bc6391707c5a2,6.98,6.98
16230,2022-03-03 15:25:45,0x2caf9244952f121fb55689b41cf0c73e3c1dea37a708bc8d3807261c73d4736d,0x72121d60b0e2f01c0fb7fe32ca24021b42165a40,0x0c9064761eac5972f53d199fdd3bc6391707c5a2,6.97,6.97
16226,2022-03-03 15:25:45,0x2b83431e7f95d9030eaa39faab569d6635d10db666cbb1e8200bb44e0472c341,0x72121d60b0e2f01c0fb7fe32ca24021b42165a40,0x0c9064761eac5972f53d199fdd3bc6391707c5a2,6.95,6.95


In [235]:
#Balance Staked
print(f"Actualmente tiene {token_tx['stake_balance'].sum()} tokens stakeados")

#Tokens holds
print(f"y holdea {move_balance.sum() + token_tx['stake_balance'].sum()} tokens.")

Actualmente tiene -495781149.7199999 tokens stakeados
y holdea -17422795222.270004 tokens.
